In [57]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
closes = pd.read_csv('closes.csv')
closes['Date'] = pd.to_datetime(closes['Date'])
closes = closes.set_index('Date')
closes = closes[['AAPL', 'GOOGL', 'META', 'NFLX', 'NVDA']]
tickers = closes.columns

In [3]:
closes

,AAPL,GOOGL,META,NFLX,NVDA
Date,,,,,
2023-01-03,123.768456,88.899872,124.365547,294.950012,14.305582
2023-01-04,125.045029,87.862434,126.987663,309.410004,14.739296
2023-01-05,123.718987,85.987083,126.558952,309.700012,14.255614
2023-01-06,128.271118,87.124260,129.629700,315.549988,14.849224
2023-01-09,128.795593,87.802582,129.081345,315.170013,15.617717
...,...,...,...,...,...
2023-12-22,192.656174,141.140518,352.329163,486.760010,48.819527
2023-12-26,192.108856,141.170441,353.764832,491.190002,49.268425
2023-12-27,192.208359,140.023270,356.755859,491.790009,49.406395


In [4]:
ret = closes.pct_change().dropna()
mu = ret.mean()
mu

AAPL     0.001835
GOOGL    0.001990
META     0.004493
NFLX     0.002288
NVDA     0.005442
dtype: float64

In [5]:
sigma = ret.cov()
sigma

,AAPL,GOOGL,META,NFLX,NVDA
AAPL,0.000158,0.000127,0.000170,0.000125,0.000171
GOOGL,0.000127,0.000367,0.000292,0.000146,0.000237
META,0.000170,0.000292,0.000630,0.000192,0.000315
NFLX,0.000125,0.000146,0.000192,0.000561,0.000221
NVDA,0.000171,0.000237,0.000315,0.000221,0.000932


In [6]:
sigma_inv = np.linalg.inv(sigma)
pd.DataFrame(sigma_inv)

,0,1,2,3,4
0,11096.807698,-1679.987062,-1437.333444,-1222.955293,-829.963798
1,-1679.987062,4870.328808,-1563.879458,-215.587378,-353.372486
2,-1437.333444,-1563.879458,2888.693856,-151.444650,-277.813020
3,-1222.955293,-215.587378,-151.444650,2244.203515,-201.882636
4,-829.963798,-353.372486,-277.813020,-201.882636,1456.919157


In [7]:
ones = np.ones(len(mu))
ones

array([1., 1., 1., 1., 1.])

In [8]:
w = (sigma_inv.dot(ones))/(ones.T.dot(sigma_inv).dot(ones))
w

array([ 0.88608136,  0.15810722, -0.08100105,  0.0676284 , -0.03081593])

In [9]:
rp = w.T.dot(mu)
rp

0.0015635377395979218

In [10]:
mu

AAPL     0.001835
GOOGL    0.001990
META     0.004493
NFLX     0.002288
NVDA     0.005442
dtype: float64

In [11]:
U = np.array([mu, ones]).T
U

array([[0.00183493, 1.        ],
       [0.00198971, 1.        ],
       [0.00449317, 1.        ],
       [0.00228751, 1.        ],
       [0.0054418 , 1.        ]])

In [12]:
M = U.T.dot(sigma_inv).dot(U)
pd.DataFrame(M)

,0,1
0,0.047738,10.457745
1,10.457745,6688.514586


In [13]:
M_inv = np.linalg.inv(M)
pd.DataFrame(M_inv)

,0,1
0,31.860075,-0.049814
1,-0.049814,0.000227


In [14]:
u = np.array([[rp],
              [1]])
u

array([[0.00156354],
       [1.        ]])

In [15]:
w2 = sigma_inv.dot(U).dot(M_inv).dot(u)
w2

array([[ 0.88608136],
       [ 0.15810722],
       [-0.08100105],
       [ 0.0676284 ],
       [-0.03081593]])